In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Input, Flatten, LeakyReLU, Dropout, Conv2DTranspose, Reshape
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.initializers import RandomNormal

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from glob import glob
import os, sys
import shutil
from PIL import Image
from numpy import asarray
import logging
from IPython.core.pylabtools import figsize


In [2]:
# batch, epoch, image output period
batch_size = 64
epochs = 1000
sample_period = 5

# image_size and latent_dim
image_size = (64,64,3)
H = image_size[0]
W = image_size[1]
C = image_size[2]
D = H*W*C
latent_dim = 100

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
# !mkdir data
!unzip -q /content/drive/MyDrive/Coding/Data/archive.zip

In [5]:
img_path = glob("img_align_celeba/img_align_celeba/*.jpg")
train_path = "img_align_celeba/"


In [6]:
data_gen = image_dataset_from_directory(directory=train_path, batch_size= batch_size, image_size=(H,W), label_mode=None, shuffle= True)

data_gen = data_gen.map(lambda x : ((x/255.0)-0.5)*2)

Found 202599 files belonging to 1 classes.


흑백사진 조심하기

In [7]:
def build_generator(latent_dim):
  model = Sequential()
  model.add(Input(shape=(latent_dim,)))
  model.add(Dense(256,))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dense(8192))
  
  
  # reshape
  model.add(tf.keras.layers.Reshape((4,4,512)))

  
  model.add(Conv2DTranspose(256,kernel_size=(4,4), strides=(2,2), activation=LeakyReLU(alpha = 0.2), padding="same"))
  model.add(Conv2DTranspose(128,kernel_size=(4,4), strides=(2,2), activation=LeakyReLU(alpha = 0.2), padding="same"))
  
  model.add(Conv2DTranspose(64,kernel_size=(4,4), strides=(2,2), activation= LeakyReLU(alpha = 0.2), padding="same"))
  

  model.add(Conv2DTranspose(3,kernel_size=(3,3), strides = (2,2), activation= "tanh", padding="same"))
  
  return model

In [8]:
def build_discriminiator(img_size = (H,W,C)):
  model = Sequential()
  model.add(Conv2D(64, 4, strides=2, padding='same', input_shape = img_size))
  model.add(LeakyReLU(alpha=0.2))
  

  model.add(Conv2D(128, 4, strides=2, padding="same"))
  model.add(LeakyReLU(alpha = 0.3))
  
  
  model.add(Conv2D(256, 4, strides=2, padding = "same"))
  model.add(LeakyReLU(alpha = 0.3))

  model.add(Conv2D(512, 4, strides=2, padding = "same"))
  model.add(LeakyReLU(alpha = 0.3))
  
  
  
  model.add(Flatten())
  # model.add(Dropout(0.3))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha = 0.3))
  model.add(Dense(64))
  model.add(LeakyReLU(alpha = 0.3))
  model.add(Dense(16))
  model.add(LeakyReLU(alpha = 0.3))
  model.add(Dense(1, activation = "sigmoid"))
  
  return model

In [9]:
def build_gan(discriminator, generator):
    discriminator.trainable = False
    gan_input = Input(shape = (latent_dim,))
    generator_output = generator(gan_input)
    gan_output = discriminator(generator_output)

    gan = Model(inputs = gan_input, outputs = gan_output)

    gan.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer = Adam(learning_rate = 0.0001, beta_1 = 0.5), metrics=["accuracy"])

    gan.summary()

    discriminator.trainable = True

    return gan

In [10]:
discriminator = build_discriminiator(img_size=(H,W,C))
discriminator.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer= Adam(learning_rate = 0.0001, beta_1 = 0.5), metrics=["accuracy"])

generator = build_generator(latent_dim)
gan = build_gan(discriminator=discriminator, generator=generator)



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential_1 (Sequential)   (None, 64, 64, 3)         7114627   
                                                                 
 sequential (Sequential)     (None, 1)                 6985249   
                                                                 
Total params: 14,099,876
Trainable params: 7,114,627
Non-trainable params: 6,985,249
_________________________________________________________________


In [11]:
ones = np.ones((batch_size,1))
zeros = np.zeros((batch_size,1))

In [12]:
d_losses = []
g_losses = []

In [13]:
file_path = "/content/drive/MyDrive/Coding/image_result/CelebA"
if not os.path.exists(file_path):
  os.makedirs(file_path)

In [14]:
check_path = "/content/drive/MyDrive/Coding/Check_point/CelebA"
if not os.path.exists(check_path):
  os.makedirs(check_path)
check_point = tf.train.Checkpoint(generator = generator, discriminator = discriminator, gan=gan, step = tf.Variable(1))
check_point_manager = tf.train.CheckpointManager(checkpoint=check_point, directory= check_path, max_to_keep= 5)

In [15]:
check_point.restore(check_point_manager.latest_checkpoint)

In [18]:
def sample_images(epoch):
  rows, cols = 3,3
  
  fig, ax = plt.subplots(rows,cols,figsize = (10,10))
  fig.tight_layout()
  noise = tf.random.normal([rows*cols, latent_dim])

  imgs = generator.predict(noise)
  imgs = (imgs + 1)/2.0

  counter = 0
  for i in range(rows):
    for j in range(cols):
      ax[i][j].imshow(imgs[counter])
      plt.axis("off")
      counter += 1

  plt.savefig(file_path + f"/%d.png" %epoch)
  plt.close()

In [20]:
for epoch in range(epochs):
    for jj in data_gen:
        real_img = jj.numpy().astype("float32")

        if len(real_img) < batch_size:
            continue

        noise = tf.random.normal([batch_size, latent_dim])

        fake_img = generator(noise)

        fake_loss, fake_acc = discriminator.train_on_batch(fake_img, zeros)
        real_loss, real_acc = discriminator.train_on_batch(real_img, ones)

        d_loss = fake_loss + real_loss
        d_acc = fake_acc + real_acc

        g_loss, g_acc = gan.train_on_batch(noise, ones)

        d_losses.append(d_loss)
        g_losses.append(g_loss)

    print(f"epoch: {epoch + 1 }/{epochs}  d_loss : {d_loss:.2f} , d_acc: {d_acc:.2f}, g_loss : {g_loss:.2f},  g_acc : {g_acc:.2f}") 

    if epoch % sample_period == 0:
        sample_images(epoch)
        check_point.step.assign_add(1)
        check_point_manager.save()




        